In [1]:
!pip install -r requirement.txt

In [62]:
import numpy as np
import geatpy as ea
import pandas as pd
import random
from datetime import datetime
pd.set_option('display.max_columns', 500) # show all columns
print(ea.__version__)

debug = 1 # debug mode

2.7.0


### PARAMETER INPUT

TODO: check input validation

The elements in list must be in order.

In [73]:
# boolen
NbvCost = False # True: max nbv; False: min cost

# float
minTotalNbv = 100000
maxTotalNbv = 10000000

# float
minTotalCost = 0
maxTotalCost = 0

# float
fleetAgeLowBound = [3, 6, 9]
fleetAgeUpBound = [6, 9, 12]
fleetAgeLimit = [0.3, 0.2, 0.4]

# float
OnHireLimit = 1.0

# float
weightedAgeLowBound = [2.5, 4, 4.5]
weightedAgeUpBound = [3.6, 4.5, 5.2]
weightedAgeLimit = [0.3, 0.4, 0.2]

# string
productType = ['D40', 'D20', 'R40']
# float
productLimit = [0.1, 0.1, 0.1]

# string
lesseeType = ['MSC', 'ONE', 'HAPAG']
# float
lesseeLimit = [0.1, 0.1, 0.1]

# string
contractType = ['LE', 'LF', 'LT']
# float
contractLimit = [0.1, 0.1, 0.1]

### READ DATA
TODO: read data from GP

In [3]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')

In [10]:
# randomly choose 1000 samples
n = 10000
rawData = rawData.iloc[:30000, :61]

In [11]:
data = rawData.sample(n)
print(data.shape)
data.sample(3)

(10000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,...,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
11733,FZOPS,DF3RM,FFAU3700818,4420.0,DFIC-FL-2104,3/3/2021,1691378.0,DFC3,D4H,FFAU3700818,...,1.7,2021 NP,Reasonable,3.520164,10.423562,from ANZ portfolio,LE-ONE-09BD4H,NaN,1499.0,1.0
25909,FZOPS,DF3RM,FBIU0321246,2262.0,CIMC-FL-1804,3/21/2018,1182470.0,DFC3,D20,FBIU0321246,...,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0
22488,FZOPS,DF3RM,FBIU0343543,2267.0,CIMC-FL-1804,3/22/2018,1184700.0,DFC3,D20,FBIU0343543,...,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0


### DATA PREPARATION

In [12]:
input_data = \
    data.loc[:, ['Cost', 'Mfr Dt Fz', 'Product', 'Contract Cust Id', 'Contract Dt', 'Contract Expiration Dt', \
                 'Nbv', 'Billing Status Fz', 'Fleet Year Fz', 'Age x CEU', 'Contract Lease Type']]
input_data.head(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,Contract Lease Type
27938,3682.0,1/3/2018,D4H,HAPAG,2017-11-01,2030-01-31,2869.540,ON,4.49,7.633,LF
2280,3832.0,12/18/2017,D4H,HAPAG,2017-11-01,2030-01-31,2981.832,ON,4.53,7.701,LF
2455,3832.0,12/18/2017,D4H,HAPAG,2017-11-01,2030-01-31,2981.794,ON,4.53,7.701,LF


#### Container Age

Assign to `FleetAge1`, `FleetAge2`, `FleetAge3`

In [40]:
def SelectFleetAge(age, i):
    if fleetAgeLowBound[i] <= age < fleetAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

input_data['FleetAge1'] = input_data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 0), axis=1)
input_data['FleetAge2'] = input_data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 1), axis=1)
input_data['FleetAge3'] = input_data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 2), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,Contract Lease Type,FleetAge1,FleetAge2,FleetAge3,Status
16150,6000.0,5/9/2021,D4H,MSC,2021-04-01,2029-06-30,5612.350,ON,1.14,1.938,LT,0,0,0,1
29529,2182.0,1/2/2018,D20,HAPAG,2017-11-01,2030-01-31,1784.489,ON,4.49,4.490,LF,1,0,0,1
9556,4420.0,3/24/2021,D4H,ONE,2020-11-16,2029-05-15,4136.340,ON,1.27,2.159,LT,0,0,0,1


#### Status

Assign new column `Status`

In [41]:
def SelectStatus(status):
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

input_data['Status'] = input_data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,Contract Lease Type,FleetAge1,FleetAge2,FleetAge3,Status
4689,3682.0,2/28/2018,D4H,MSC,2018-03-01,2032-07-31,2933.007,ON,4.33,7.361,LT,1,0,0,1
29631,2182.0,1/2/2018,D20,HAPAG,2017-11-01,2030-01-31,1784.490,ON,4.49,4.490,LF,1,0,0,1
10606,4420.0,3/24/2021,D4H,ONE,2020-11-16,2029-05-15,4136.340,ON,1.27,2.159,LT,0,0,0,1


#### Weighted Age

Assign new column `WeightedAge1`, `WeightedAge2`, `WeightedAge3`

In [42]:
def SelectWeightedAge(age, i):
    if weightedAgeLowBound[i] <= age < weightedAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

input_data['WeightedAge1'] = input_data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 0), axis=1)
input_data['WeightedAge2'] = input_data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 1), axis=1)
input_data['WeightedAge3'] = input_data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 2), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,Contract Lease Type,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3
29619,2182.0,1/2/2018,D20,HAPAG,2017-11-01,2030-01-31,1784.49,ON,4.49,4.490,LF,1,0,0,1,0,1,0
26815,3682.0,1/1/2018,D4H,HAPAG,2017-11-01,2030-01-31,2869.54,ON,4.49,7.633,LF,1,0,0,1,0,0,0
10105,4420.0,3/24/2021,D4H,ONE,2020-11-16,2029-05-15,4136.34,ON,1.27,2.159,LT,0,0,0,1,0,0,0


#### Product

Assign new column `ProductType1`, `ProductType2`, `ProductType3`

In [45]:
def SelectProductType(product, i):
    if product == productType[i]:
        product = 1
    else:
        product = 0
    return product

input_data['ProductType1'] = input_data.apply(lambda x: SelectProductType(x['Product'], 0), axis=1)
input_data['ProductType2'] = input_data.apply(lambda x: SelectProductType(x['Product'], 1), axis=1)
input_data['ProductType3'] = input_data.apply(lambda x: SelectProductType(x['Product'], 2), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,...,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3
29876,2182.0,1/2/2018,D20,HAPAG,2017-11-01,2030-01-31,1784.490,ON,4.49,4.490,...,1,0,0,1,0,1,0,0,1,0
13400,6007.0,4/28/2021,D4H,MSC,2021-04-01,2029-06-30,5618.800,ON,1.17,1.989,...,0,0,0,1,0,0,0,0,0,0
1493,3832.0,12/18/2017,D4H,HAPAG,2017-11-01,2030-01-31,2981.838,ON,4.53,7.701,...,1,0,0,1,0,0,0,0,0,0


#### Lessee

Assign new column `Lessee1`, `Lessee2`, `Lessee3`

In [50]:
def SelectLessee(lessee, i):
    if lessee == lesseeType[i]:
        lessee = 1
    else:
        lessee = 0
    return lessee

input_data['Lessee1'] = input_data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 0), axis=1)
input_data['Lessee2'] = input_data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 1), axis=1)
input_data['Lessee3'] = input_data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 2), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,...,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3
12770,4420.0,3/3/2021,D4H,ONE,2020-11-16,2029-05-15,4117.430,ON,1.32,2.244,...,1,0,0,0,0,0,0,0,1,0
15756,6000.0,4/8/2021,D4H,MSC,2021-04-01,2029-06-30,5612.350,ON,1.23,2.091,...,1,0,0,0,0,0,0,1,0,0
18829,5150.0,4/20/2021,D4H,MSC,2021-04-01,2029-06-30,4773.958,ON,1.19,2.023,...,1,0,0,0,0,0,0,1,0,0


#### Contract Type

Assign new column `ContractType1`, `ContractType2`, `ContractType3`

In [57]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

input_data['ContractType1'] = input_data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 0), axis=1)
input_data['ContractType2'] = input_data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 1), axis=1)
input_data['ContractType3'] = input_data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 2), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,...,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3,ContractType1,ContractType2,ContractType3
12253,4420.0,3/3/2021,D4H,ONE,2020-11-16,2029-05-15,4117.43,ON,1.32,2.244,...,0,0,0,0,0,1,0,0,0,1
27690,3682.0,1/1/2018,D4H,HAPAG,2017-11-01,2030-01-31,2869.54,ON,4.49,7.633,...,0,0,0,0,0,0,1,0,1,0
12840,4420.0,3/3/2021,D4H,ONE,2020-11-16,2029-05-15,4117.43,ON,1.32,2.244,...,0,0,0,0,0,1,0,0,0,1


### SAVE DATA

In [58]:
input_data.to_csv('prepared_data_demo.csv')

In [64]:
prepared_data = pd.read_csv('./prepared_data_demo.csv')
prepared_data.head(3)

,Unnamed: 0,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,Contract Lease Type,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3,ContractType1,ContractType2,ContractType3
0,27938,3682.0,1/3/2018,D4H,HAPAG,2017-11-01,2030-01-31,2869.540,ON,4.49,7.633,LF,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,2280,3832.0,12/18/2017,D4H,HAPAG,2017-11-01,2030-01-31,2981.832,ON,4.53,7.701,LF,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
2,2455,3832.0,12/18/2017,D4H,HAPAG,2017-11-01,2030-01-31,2981.794,ON,4.53,7.701,LF,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0


### Model Part

In [70]:
nbv = prepared_data['Nbv'].to_numpy().reshape(n, 1)
cost = prepared_data['Cost'].to_numpy().reshape(n, 1)
fleetAge = np.stack([prepared_data['FleetAge1'].to_numpy(),
                     prepared_data['FleetAge2'].to_numpy(),
                     prepared_data['FleetAge3'].to_numpy()])
status = prepared_data['Status'].to_numpy()
weightedAge = np.stack([prepared_data['WeightedAge1'].to_numpy(),
                        prepared_data['WeightedAge2'].to_numpy(),
                        prepared_data['WeightedAge3'].to_numpy()])
productType = np.stack([prepared_data['ProductType1'].to_numpy(),
                        prepared_data['ProductType2'].to_numpy(),
                        prepared_data['ProductType3'].to_numpy()])
lessee = np.stack([prepared_data['Lessee1'].to_numpy(),
                   prepared_data['Lessee2'].to_numpy(),
                   prepared_data['Lessee3'].to_numpy()])
contractType = np.stack([prepared_data['ContractType1'].to_numpy(),
                         prepared_data['ContractType2'].to_numpy(),
                         prepared_data['ContractType3'].to_numpy(),])
if debug:
    print('nbv shape: ', nbv.shape)
    print('cost shape:', cost.shape)
    print('fleetAge shape: ', fleetAge.shape)
    print('status shape:', status.shape)
    print('weightedAge shape:', weightedAge.shape)
    print('productType shape:', productType.shape)
    print('lessee shape:', lessee.shape)
    print('contractType shape:', contractType.shape)

nbv shape:  (10000, 1)
cost shape: (10000, 1)
fleetAge shape:  (3, 10000)
status shape: (10000,)
weightedAge shape: (3, 10000)
productType shape: (3, 10000)
lessee shape: (3, 10000)
contractType shape: (3, 10000)


#### Constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. minTotalNbv <= Total NBV <= maxTotalNbv
    
2. minTotalCost <= Total Cost <= maxTotalCost
    
3. Container Age
    
4. Status:

    1. OnHire >= OnHireLimit

5. Weighted Age


6. Product
    
7. Lessee
    
8. Contract Type


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem
Define objective function and constraint functions

In [74]:
def DEMO():
    class MyProblem(ea.Problem):
        def __init__(self):
            name ='MyProblem'
            M = 1
            maxormins = [-1 if NbvCost else 1] # -1: max, 1: min
            Dim = n # num of variables
            varTypes = [1] * Dim # integers: 0 or 1
            lb = [0] * Dim # low bound
            ub = [1] * Dim # up bound
            lbin = [1] * Dim # low bound include
            ubin = [1] * Dim # up bound include
            ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb,ub, lbin, ubin)
            
        def aimFunc(self, pop):
            Vars = pop.Phen
            
            # objective function
            pop.ObjV = Vars @ nbv if NbvCost else Vars @ cost

            # constraints functions
            pop.CV = np.hstack([
                -(Vars @ nbv - minTotalNbv),
                Vars @ nbv - maxTotalNbv
    
            ])




    problem = MyProblem()
    Encoding ='RI'
    NIND = 100 # init number of population instance
    Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges,problem.borders)
    population = ea.Population(Encoding, Field, NIND)

    # set parameters
    myAlgorithm = ea.soea_DE_best_1_L_templet(problem, population)
    myAlgorithm.MAXGEN = 10000
    myAlgorithm.mutOper.F = 0.5
    myAlgorithm.recOper.XOVR = 0.7 # probability of chrom crossover
    myAlgorithm.logTras = 1 # record log each logTras generations
    myAlgorithm.trappedValue = 1e-6
    myAlgorithm.maxTrappedCount = 100 # TODO: increase it or remove it

    # TODO: random seed has a significant influence on final result!
    # outputMsg=True to show the result
    res = ea.optimize(myAlgorithm, seed=2049, verbose=False, drawing=1, outputMsg=True, drawLog=False, saveFlag=False, dirName='result')
    if debug:
        print(res)
    return res

In [ ]:
# 1000, 1e-6, 100
selectedPackage = DEMO()

`NIND` barely has any influence on the final result, but will slow down the process. -- what about reduce to 100

`maxTrappedCount` has a big influence on the final result. U should try to increase maxTrappedCount to gain better performance.

#### Output

1000 gen = 13 minutes
No early stop if no feasible result is found.


In [53]:
print('Time Consumption: ', np.round(selectedPackage['executeTime'], decimals=2), 's')
print(selectedPackage['stopMsg'])

if selectedPackage['success']:
    print('Algorithm Succeeded!')
    print('===================================================================================================')
    print('Optimal Target Value: ', np.round(selectedPackage['ObjV'].item(), decimals=2))
    print('Constraint Violation: ', np.round(selectedPackage['CV'][0], decimals=2))
    if debug:
        # TODO: doing
        print('===================================================================================================')
        Vars = selectedPackage['Vars'][0]
        print('NBV of standard equip:', np.round((Vars * product @ nbv) / (Vars @ nbv), decimals=2)[0])
        print('Avg age of Manufacturing:', np.round((Vars @ mfrAge) / Vars.sum(), decimals=2)[0])
        print('Avg unexpired term under leases:', np.round((Vars * lesseStatus @ expYear) / (Vars * lesseStatus).sum(), decimals=2)[0])
        print('Top 1 NBV:', np.round((Vars * top1 @ nbv) / (Vars @ nbv), decimals=2)[0])
        print('Top 3 NBV:', np.round((Vars * top3 @ nbv) / (Vars @ nbv), decimals=2)[0])
        
else:
    print('No Feasible Results...')

Time Consumption:  318.33 s
The algotirhm stepped because it exceeded the trapped count limit.
Algorithm Succeeded!
Optimal Target Value:  19458029.26
Constraint Violation:  [-0.2  -3.44 -1.65 -0.  ]
NBV of standard equip: 1.0
Avg age of Manufacturing: 3.56
Avg unexpired term under leases: 6.65
Top 1 NBV: 0.5
Top 3 NBV: 0.98


In [54]:
# print final result
print('Selected Package:')
print('===================================================================================================')
print(len(selectedPackage['Vars'].nonzero()[1]), 'containers selected.')
print(selectedPackage['Vars'].nonzero()[1]) # index begins from 0
print('===================================================================================================')
print(selectedPackage['Vars'][0].tolist())

Selected Package:
4943 containers selected.
[   0    3    4 ... 9992 9996 9999]
[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0